In [1]:
import pandas as pd

data = pd.read_csv('WDI_Data.csv')
    

data['21st'] = data.ix[:, '2000':].mean(axis=1)
print sum(data['21st'].isnull())

# data[data['Country Name']=='Caribbean small states'].head()

104967


In [2]:
grouped = data.groupby('Indicator Name')

def one_year(year='21st'):
    '''Return a dataframe with countries as rows, indicators as columns, all for one (year) selection'''
    indic_dict = {}
    for indicator, group in grouped:
        for index, row in group.iterrows():
            if indicator not in indic_dict:
                indic_dict[indicator] = []

            indic_dict[indicator].append(row[year])

    for indicator, group in grouped:
        names = group['Country Name'].tolist()
        break   
        
    return pd.DataFrame(indic_dict, index=names) 

In [8]:
df = one_year()

In [9]:
# print(__doc__)
from time import time

import numpy as np
from scipy import ndimage
from matplotlib import pyplot as plt

from sklearn import manifold, datasets

# digits = datasets.load_digits(n_class=10)
# X = digits.data
# y = digits.target
# n_samples, n_features = X.shape

features = {'Simple': ['Population, total', 'CO2 emissions (kt)', 'Net migration', 'Trade (% of GDP)'],
            'Education': [],
            'Health': []}

sample = df[features['Simple']].dropna()
countries = sample.index.tolist()

X = sample.as_matrix()
n_samples, n_features = X.shape

print n_samples, n_features
np.random.seed(0)


# def nudge_images(X, y):
#     # Having a larger dataset shows more clearly the behavior of the
#     # methods, but we multiply the size of the dataset only by 2, as the
#     # cost of the hierarchical clustering methods are strongly
#     # super-linear in n_samples
#     shift = lambda x: ndimage.shift(x.reshape((8, 8)),
#                                   .3 * np.random.normal(size=2),
#                                   mode='constant',
#                                   ).ravel()
#     X = np.concatenate([X, np.apply_along_axis(shift, 1, X)])
#     Y = np.concatenate([y, y], axis=0)
#     return X, Y


# X, y = nudge_images(X, y)


#----------------------------------------------------------------------
# Visualize the clustering
# def plot_clustering(X_red, X, labels, title=None):
#     x_min, x_max = np.min(X_red, axis=0), np.max(X_red, axis=0)
#     X_red = (X_red - x_min) / (x_max - x_min)

#     plt.figure(figsize=(6, 4))
#     for i in range(X_red.shape[0]):
#         plt.text(X_red[i, 0], X_red[i, 1], str(y[i]),
#                  %colorsor=plt.cm.spectral(labels[i] / 10.),
#                  fontdict={'weight': 'bold', 'size': 9})

#     plt.xticks([])
#     plt.yticks([])
#     if title is not None:
#         plt.title(title, size=17)
#     plt.axis('off')
#     plt.tight_layout()

#----------------------------------------------------------------------
# 2D embedding of the digits dataset
# print("Computing embedding")
X_red = manifold.SpectralEmbedding(n_components=2).fit_transform(X)
# print("Done.")

from sklearn.cluster import AgglomerativeClustering

def cluster_label_dict(countries, clustering):
    '''Return a dictionary with clustering labels as keys and a list of labeled countries as the value'''
    label_dict = {}
    for index, label in enumerate(clustering.labels_):
        if label not in label_dict:
            label_dict[label] = []    
        label_dict[label].append(countries[index])
    return label_dict
    
#Perform agglomerative clustering with three different linkages
for linkage in ('ward', 'average', 'complete'):
    clustering = AgglomerativeClustering(linkage=linkage, n_clusters=6)
    t0 = time()
    clustering.fit(X_red)
    print("%s : %.2fs" % (linkage, time() - t0))
    clust_dict = cluster_label_dict(countries, clustering)
    print clust_dict
#     plot_clustering(X_red, X, clustering.labels_, "%s linkage" % linkage)


plt.show()


217 4
ward : 0.00s
{0: ['Other small states', 'Small states', 'Afghanistan', 'Algeria', 'Angola', 'Argentina', 'Australia', 'Burkina Faso', 'Cambodia', 'Cameroon', 'Canada', 'Chile', "Cote d'Ivoire", 'Ecuador', 'Ghana', 'Guatemala', 'Iraq', 'Kazakhstan', 'Kenya', 'Madagascar', 'Malawi', 'Malaysia', 'Mali', 'Morocco', 'Mozambique', 'Nepal', 'Netherlands', 'Niger', 'Peru', 'Poland', 'Romania', 'Saudi Arabia', 'Sri Lanka', 'Sudan', 'Syrian Arab Republic', 'Uganda', 'Uzbekistan', 'Venezuela, RB', 'Yemen, Rep.', 'Zambia', 'Zimbabwe'], 1: ['Antigua and Barbuda', 'Aruba', 'Bahamas, The', 'Bahrain', 'Barbados', 'Belize', 'Bhutan', 'Brunei Darussalam', 'Cabo Verde', 'Comoros', 'Cyprus', 'Djibouti', 'Equatorial Guinea', 'Estonia', 'Fiji', 'French Polynesia', 'Gabon', 'Gambia, The', 'Grenada', 'Guinea-Bissau', 'Guyana', 'Iceland', 'Kiribati', 'Luxembourg', 'Macao SAR, China', 'Maldives', 'Malta', 'Mauritius', 'Montenegro', 'Qatar', 'Samoa', 'Seychelles', 'Solomon Islands', 'St. Lucia', 'St. Vince